In [1]:
pip install numpy tensorflow keras scikit-learn matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Image parameters
img_height = 224
img_width = 224
batch_size = 32

# Prepare dataset with train-test split
def prepare_dataset(fire_path, non_fire_path):
    fire_images = [os.path.join(fire_path, f) for f in os.listdir(fire_path) if f.endswith(('.jpg', '.png'))]
    non_fire_images = [os.path.join(non_fire_path, f) for f in os.listdir(non_fire_path) if f.endswith(('.jpg', '.png'))]
    
    # Labels
    fire_labels = [1] * len(fire_images)
    non_fire_labels = [0] * len(non_fire_images)
    
    # Combine datasets
    all_images = fire_images + non_fire_images
    all_labels = fire_labels + non_fire_labels
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        all_images, all_labels, test_size=0.2, random_state=42, stratify=all_labels
    )
    
    return X_train, X_test, y_train, y_test

# Image data generator
def create_data_generator(images, labels, augment=True):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20 if augment else 0,
        width_shift_range=0.2 if augment else 0,
        height_shift_range=0.2 if augment else 0,
        shear_range=0.2 if augment else 0,
        zoom_range=0.2 if augment else 0,
        horizontal_flip=augment
    )
    
    # Convert images and labels to numpy arrays
    images_array = []
    for img_path in images:
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        images_array.append(img_array)
    
    images_array = np.array(images_array)
    labels_array = np.array(labels)
    
    return datagen.flow(images_array, labels_array, batch_size=batch_size)

# Create CNN model with dropout and regularization
def create_fire_detection_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001), input_shape=(img_height, img_width, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        layers.MaxPooling2D((2, 2)),
        
        layers.Flatten(),
        layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Prepare dataset
X_train, X_test, y_train, y_test = prepare_dataset('fire_images', 'non_fire_images')

# Create data generators
train_generator = create_data_generator(X_train, y_train, augment=True)
test_generator = create_data_generator(X_test, y_test, augment=False)

# Define callbacks to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True
)

lr_reducer = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=5, 
    min_lr=1e-6
)

# Create and train model
model = create_fire_detection_model()
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=50,
    callbacks=[early_stopping, lr_reducer]
)

# Evaluate model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy*100:.2f}%')

# Save model
model.save('fire_detection_model.h5')

# Prediction function
def predict_fire(image_path):
    img = tf.keras.preprocessing.image.load_img(
        image_path, 
        target_size=(img_height, img_width)
    )
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    
    prediction = model.predict(img_array)
    return 'Fire Detected' if prediction[0] > 0.5 else 'No Fire'

In [6]:
predict_fire('/kaggle/input/forest-fire-bigdata/Forest_Fire_BigData/non_fire_images/abc191.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


'No Fire'